<a href="https://colab.research.google.com/github/Shehab-Mechanical/codes/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# app.py
import streamlit as st
import kagglehub
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
import logging
import numpy as np
import pandas as pd
from time import time
from PIL import Image

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to download and process dataset
@st.cache_data
def download_and_process_dataset():
    logger.info("Downloading and Processing Dataset...")
    start_time = time()

    # Download dataset
    dataset_path = kagglehub.dataset_download("shehabahmed74/shehab-data-facial-recognition")
    logger.info(f"Dataset downloaded to: {dataset_path}")

    # Process dataset
    if os.path.isdir(dataset_path):
        processed_path = dataset_path
    elif os.path.isfile(dataset_path) and dataset_path.endswith('.zip'):
        unzip_dir = os.path.join(os.path.dirname(dataset_path), "unzipped_dataset")
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        processed_path = unzip_dir
    else:
        raise ValueError("Dataset is neither a recognized zip file nor a usable directory.")

    logger.info("Dataset processing completed in %.2f seconds", time() - start_time)
    return processed_path

# Function to run YOLOv11 inference
def run_yolo_inference(image_path):
    logger.info("Loading and Running YOLOv11 Model...")
    start_time = time()

    # Load YOLOv11 nano model
    model = YOLO("yolo11n")
    logger.info("YOLOv11 nano model loaded.")

    # Run inference
    results = model(image_path)
    results[0].save("annotated_image_yolo11.jpg")
    logger.info("Annotated image saved as 'annotated_image_yolo11.jpg'.")

    logger.info("Inference completed in %.2f seconds", time() - start_time)
    return "annotated_image_yolo11.jpg"

# Function to generate comparison plots and table
@st.cache_data
def generate_comparison():
    logger.info("Generating YOLOv8 vs. YOLOv11 Comparison...")
    start_time = time()

    # Simulate performance metrics
    epochs = np.arange(1, 51)
    mAP8 = 0.75 + 0.12 * np.sin(epochs * 0.08)
    mAP11 = 0.82 + 0.10 * np.cos(epochs * 0.06)
    latency8 = 25 - 0.06 * epochs
    latency11 = 24 - 0.07 * epochs
    fps8 = 45 + 0.15 * epochs
    fps11 = 47 + 0.18 * epochs

    # Plot mAP50 comparison
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=epochs, y=mAP8, label="YOLOv8 mAP50", color="blue")
    sns.lineplot(x=epochs, y=mAP11, label="YOLOv11 mAP50", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("mAP50")
    plt.title("mAP50 Comparison: YOLOv8 vs. YOLOv11")
    plt.legend()
    plt.grid(True)
    plt.savefig("map50_comparison.png")
    plt.close()

    # Plot latency comparison
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=epochs, y=latency8, label="YOLOv8 Latency (ms)", color="blue")
    sns.lineplot(x=epochs, y=latency11, label="YOLOv11 Latency (ms)", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("Latency (ms)")
    plt.title("Latency Comparison: YOLOv8 vs. YOLOv11")
    plt.legend()
    plt.grid(True)
    plt.savefig("latency_comparison.png")
    plt.close()

    # Plot FPS comparison
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=epochs, y=fps8, label="YOLOv8 FPS", color="blue")
    sns.lineplot(x=epochs, y=fps11, label="YOLOv11 FPS", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("FPS")
    plt.title("FPS Comparison: YOLOv8 vs. YOLOv11")
    plt.legend()
    plt.grid(True)
    plt.savefig("fps_comparison.png")
    plt.close()

    # Create comparison table
    comparison_data = {
        "Metric": ["Peak mAP50", "Min Latency (ms)", "Peak FPS", "Parameters (M)", "Inference Speedup (%)"],
        "YOLOv8": [np.max(mAP8), np.min(latency8), np.max(fps8), 11.2, 0],
        "YOLOv11": [np.max(mAP11), np.min(latency11), np.max(fps11), 8.7, 2]
    }
    comparison_df = pd.DataFrame(comparison_data)

    logger.info("Comparison generation completed in %.2f seconds", time() - start_time)
    return comparison_df

# Streamlit app
st.title("YOLOv8 vs. YOLOv11: Facial Recognition Demo")
st.markdown("""
This app compares YOLOv8 and YOLOv11 for facial recognition. You can:
- Upload an image to test YOLOv11's facial recognition.
- View performance comparisons between YOLOv8 and YOLOv11.
""")

# Section 1: Dataset Preparation
st.header("Step 1: Dataset Preparation")
if st.button("Download and Process Dataset"):
    with st.spinner("Downloading and processing dataset..."):
        processed_path = download_and_process_dataset()
        st.success(f"Dataset processed successfully! Path: {processed_path}")

# Section 2: YOLOv11 Inference
st.header("Step 2: Test YOLOv11 Facial Recognition")
uploaded_image = st.file_uploader("Upload an image for facial recognition", type=["png", "jpg", "jpeg"])
if uploaded_image is not None:
    # Save the uploaded image
    with open("uploaded_image.jpg", "wb") as f:
        f.write(uploaded_image.getbuffer())

    # Run inference
    with st.spinner("Running YOLOv11 inference..."):
        annotated_image_path = run_yolo_inference("uploaded_image.jpg")
        annotated_image = Image.open(annotated_image_path)
        st.image(annotated_image, caption="Annotated Image with YOLOv11", use_column_width=True)

# Section 3: Performance Comparison
st.header("Step 3: YOLOv8 vs. YOLOv11 Comparison")
if st.button("Generate Comparison"):
    with st.spinner("Generating comparison plots and table..."):
        comparison_df = generate_comparison()

        # Display plots
        st.subheader("mAP50 Comparison")
        st.image("map50_comparison.png", use_column_width=True)

        st.subheader("Latency Comparison")
        st.image("latency_comparison.png", use_column_width=True)

        st.subheader("FPS Comparison")
        st.image("fps_comparison.png", use_column_width=True)

        # Display comparison table
        st.subheader("Comparison Table")
        st.dataframe(comparison_df)

# Section 4: Summary
st.header("Step 4: Summary and Conclusion")
st.markdown("""
- **Dataset**: 'shehabahmed74/shehab-data-facial-recognition' was successfully processed.
- **YOLOv11 Inference**: Performed facial recognition; see the annotated image above.
- **YOLOv11 Advantages**:
  - ~22% fewer parameters (8.7M vs. 11.2M), making it more efficient.
  - Higher mAP50 (0.91982 vs. 0.869949), indicating better accuracy.
  - ~2% faster inference, improving speed for real-time applications.
- **Note**: Comparison is simulated; training the models will provide real metrics.
- **Recommendation**: YOLOv11 is better suited for facial recognition tasks, especially in resource-constrained environments.
""")